In [1]:
!pip install pyts

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.2 MB/s eta 0:00:00


In [2]:
import numpy as np
from sklearn.utils import shuffle
import gdown

data_train_id = "1oG5FtbXgUgnYkQI-Om-MBq_n1s2dYUAt"
data_test_id = "1dViHrJP7Lm6nr_RhEaDB9A7IlOwy439s"
target_train_id = "1km9rjrYGDyQlLE_20Q4kLjJXWoXLgEA9"
target_test_id = "1N-xiJWDqQJq2UMaafuBwGg6ghRLvrpEM"

data_train_norm_id = "1sstp-w4HBoFiegK90uHiMwuOygNOKaEU"
data_test_norm_id = "1CyN22bU0iyYbgMwUVYNSJPc_GwqMCL1P"



gdown.download(id=data_train_id, output="data_train.npy", quiet=True)
gdown.download(id=data_test_id, output="data_test.npy", quiet=True)
gdown.download(id=target_train_id, output="target_train.npy", quiet=True)
gdown.download(id=target_test_id, output="target_test.npy", quiet=True)
gdown.download(id=data_train_norm_id, output="data_train_norm.npy", quiet=True)
gdown.download(id=data_test_norm_id, output="data_test_norm.npy", quiet=True)

'data_test_norm.npy'

In [3]:
# Cell 1: Imports & Data Loading
import numpy as np
from tensorflow.keras.utils import to_categorical

# — Replace these with your actual data-loading steps —
# For example, if you have .npy files:
data_train_norm = np.load('data_train_norm.npy')
data_test_norm  = np.load('data_test_norm.npy')
target_train    = np.load('target_train.npy')
target_test     = np.load('target_test.npy')


In [4]:
# Cell 2: Model Definitions
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, LSTM, Dense,
    Conv1D, BatchNormalization,
    GlobalAveragePooling1D
)

# def build_lstm(input_shape, num_classes):
#     inp = Input(shape=input_shape)
#     x   = LSTM(64)(inp)
#     out = Dense(num_classes, activation='softmax')(x)
#     return inp, out

# from tensorflow.keras.layers import Input, Dense, Flatten

# def build_encoder(input_shape, num_classes):
#     inp = Input(shape=input_shape)      # (None, timesteps, 1)
#     x   = Flatten()(inp)                # (None, timesteps * 1)
#     x   = Dense(64, activation='relu')(x)
#     x   = Dense(64, activation='relu')(x)
#     out = Dense(num_classes, activation='softmax')(x)
#     return inp, out



def build_resnet(input_shape, filters, num_classes):
    inp = Input(shape=input_shape)
    x   = Conv1D(filters, 3, padding='same', activation='relu')(inp)
    x   = BatchNormalization()(x)
    x   = Conv1D(filters, 3, padding='same', activation='relu')(x)
    x   = BatchNormalization()(x)
    x   = GlobalAveragePooling1D()(x)
    out = Dense(num_classes, activation='softmax')(x)
    return inp, out


In [5]:
# Cell 3: Hyperparameters & Scheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

NUM_CLASSES   = int(np.max([target_train.max(), target_test.max()]) + 1)
EPOCHS        = 50
BATCH_SIZE    = 16
LEARNING_RATE = 5e-3
source_idx    = 0   # which view to use

lr_schedule = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-5,
    verbose=1
)

build_models = [
    # ('lstm',    build_lstm)
    # ('encoder', build_encoder)
    ('resnet',  build_resnet)
]


In [6]:
# Cell 4: Train & Summarize Baseline Accuracies
from tensorflow.keras.models import Model

accuracies = {}

for name, build_fn in build_models:
    print(f"\n--- Training {name} on view{source_idx} ---")
    # reshape to (samples, timesteps, channels=1)
    X_tr = data_train_norm[:, :, source_idx].reshape(-1, data_train_norm.shape[1], 1)
    X_te = data_test_norm[:,  :, source_idx].reshape(-1, data_test_norm.shape[1], 1)
    y_tr = to_categorical(target_train, NUM_CLASSES)
    y_te = to_categorical(target_test,  NUM_CLASSES)

    # build model
    if name == 'resnet':
        inp, out = build_fn(X_tr.shape[1:], 32, NUM_CLASSES)
    else:
        inp, out = build_fn(X_tr.shape[1:], NUM_CLASSES)
    model = Model(inputs=inp, outputs=out)
    model.compile(
        loss='categorical_crossentropy',
        optimizer=Adam(learning_rate=LEARNING_RATE),
        metrics=['accuracy']
    )

    # fit
    hist = model.fit(
        X_tr, y_tr,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_data=(X_te, y_te),
        callbacks=[lr_schedule],
        verbose=0
    )

    val_acc = hist.history['val_accuracy'][-1]
    accuracies[name] = val_acc
    print(f"{name:7s} final val_accuracy = {val_acc:.4f}")

print("\n=== Baseline (no-transfer) Accuracies ===")
for m, a in accuracies.items():
    print(f" • {m:<7s}: {a:.2%}")



--- Training resnet on view0 ---

Epoch 8: ReduceLROnPlateau reducing learning rate to 0.0024999999441206455.

Epoch 14: ReduceLROnPlateau reducing learning rate to 0.0012499999720603228.

Epoch 20: ReduceLROnPlateau reducing learning rate to 0.0006249999860301614.

Epoch 31: ReduceLROnPlateau reducing learning rate to 0.0003124999930150807.

Epoch 36: ReduceLROnPlateau reducing learning rate to 0.00015624999650754035.

Epoch 41: ReduceLROnPlateau reducing learning rate to 7.812499825377017e-05.

Epoch 46: ReduceLROnPlateau reducing learning rate to 3.9062499126885086e-05.
resnet  final val_accuracy = 0.4741

=== Baseline (no-transfer) Accuracies ===
 • resnet : 47.41%
